In [ ]:
from taming.models.vqgan import VQModel
import argparse, os, sys, datetime, glob, importlib
from omegaconf import OmegaConf
import numpy as np
from PIL import Image
import torch
import torchvision
from torch.utils.data import random_split, DataLoader, Dataset
import pytorch_lightning as pl
from pytorch_lightning import seed_everything
from pytorch_lightning.trainer import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, Callback, LearningRateMonitor
from pytorch_lightning.utilities import rank_zero_only

from taming.data.utils import custom_collate
import matplotlib.pyplot as plt

In [ ]:
base_config_path = ["configs/custom_vqgan_eval.yaml"]

In [ ]:
def get_obj_from_str(string, reload=False):
    module, cls = string.rsplit(".", 1)
    if reload:
        module_imp = importlib.import_module(module)
        importlib.reload(module_imp)
    return getattr(importlib.import_module(module, package=None), cls)

In [ ]:
def instantiate_from_config(config):
    if not "target" in config:
        raise KeyError("Expected key `target` to instantiate.")
    return get_obj_from_str(config["target"])(**config.get("params", dict()))

In [ ]:
def log_local(self, save_dir, split, images, global_step, current_epoch, batch_idx):
    root = os.path.join(save_dir, "images", split)
    for k in images:
        grid = torchvision.utils.make_grid(images[k], nrow=4)

        grid = (grid+1.0)/2.0 # -1,1 -> 0,1; c,h,w
        grid = grid.transpose(0,1).transpose(1,2).squeeze(-1)
        grid = grid.numpy()
        grid = (grid*255).astype(np.uint8)
        filename = "{}_gs-{:06}_e-{:06}_b-{:06}.png".format(
            k,
            global_step,
            current_epoch,
            batch_idx)
        path = os.path.join(root, filename)
        os.makedirs(os.path.split(path)[0], exist_ok=True)
        Image.fromarray(grid).save(path)

In [ ]:
configs = [OmegaConf.load(cfg) for cfg in base_config_path]
config = OmegaConf.merge(*configs)

# config.model

In [ ]:
# create model
model = instantiate_from_config(config.model)

In [ ]:
# load the data of interest
data = instantiate_from_config(config.data)
# NOTE according to https://pytorch-lightning.readthedocs.io/en/latest/datamodules.html
# calling these ourselves should not be necessary but it is.
# lightning still takes care of proper multiprocessing though
data.prepare_data()
data.setup()

In [ ]:
test_data_loader = data.val_dataloader()
train_data_loader = data.train_dataloader()

In [ ]:
print(test_data_loader.__len__())
print(train_data_loader.__len__())

In [ ]:
# set the models to eval mode
model.eval()
model.loss.discriminator.eval()

# make a directory to hold qualtative eval images
output_path = "/work/pi_hzhang2_umass_edu/oyoungquist_umass_edu/taming-transformers/vqgan_eval/DeepAccident/negatives/"

if not os.path.exists(output_path):
    os.makedirs(output_path)

batch_num = 0

# iterate over the test set
for batch in test_data_loader:
    print(batch_num)
    # don't track the gradient
    with torch.no_grad():
        log_dict = model.log_images(batch)
    # plt.imshow(batch["image"][2].numpy())

    # print(log_dict["dis_diff"])
    # print(log_dict["feature_diff"].size())
    
    log_dict["inputs"] = log_dict["inputs"].permute(0,2,3,1)
    log_dict["reconstructions"] = log_dict["reconstructions"].permute(0,2,3,1)
    log_dict["feature_diff"] = log_dict["feature_diff"].permute(0,2,3,1)
    
    
    # print(log_dict["feature_diff"].size())
    fig, axes = plt.subplots(nrows=1, ncols=3, squeeze=True)
    
    axes[0].imshow(log_dict["inputs"][0].numpy())
    axes[0].axis('off')
    axes[0].set_title("Input")
    axes[1].imshow(log_dict["reconstructions"][0].numpy())
    axes[1].axis('off')
    axes[1].set_title("Recon.")
    axes[2].imshow(log_dict["feature_diff"][0].numpy())
    axes[2].axis('off')
    axes[2].set_title("Diff.")
    new_path = os.path.join(output_path, "neg_eval_sample_{:d}.png".format(batch_num))
    batch_num += 1
    fig.savefig(new_path)
    plt.close()

    fig, axes = plt.subplots(nrows=1, ncols=3, squeeze=True)
    
    axes[0].imshow(log_dict["inputs"][1].numpy())
    axes[0].axis('off')
    axes[0].set_title("Input")
    axes[1].imshow(log_dict["reconstructions"][1].numpy())
    axes[1].axis('off')
    axes[1].set_title("Recon.")
    axes[2].imshow(log_dict["feature_diff"][1].numpy())
    axes[2].axis('off')
    axes[2].set_title("Diff.")
    new_path = os.path.join(output_path, "neg_eval_sample_{:d}.png".format(batch_num))
    batch_num += 1
    fig.savefig(new_path)
    plt.close()


    fig, axes = plt.subplots(nrows=1, ncols=3, squeeze=True)
    
    axes[0].imshow(log_dict["inputs"][2].numpy())
    axes[0].axis('off')
    axes[0].set_title("Input")
    axes[1].imshow(log_dict["reconstructions"][2].numpy())
    axes[1].axis('off')
    axes[1].set_title("Recon.")
    axes[2].imshow(log_dict["feature_diff"][2].numpy())
    axes[2].axis('off')
    axes[2].set_title("Diff.")
    new_path = os.path.join(output_path, "neg_eval_sample_{:d}.png".format(batch_num))
    batch_num += 1
    fig.savefig(new_path)
    plt.close()
    # plt.show()

    if (batch_num > 5):
        break